In [36]:
#import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [37]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'class':'a-price a-text-price a-size-base'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'class':'a-price a-text-price a-size-medium apexPriceToPay'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [38]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=mobile+phone&sprefix=mobile+phone%2Caps%2C338&ref=nb_sb_ss_ts-doa-p_2_12"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [40]:
amazon_df

,title,price,rating,reviews,availability
0,Nokia G10 | Android 11 | Unlocked Smartphone |...,,3.6 out of 5 stars,281 ratings,In Stock.
1,Moto G Power | 2021 | 3-Day battery | Unlocked...,,4.4 out of 5 stars,556 ratings,In Stock.
2,Moto G Stylus 5G | 2021 | 2-Day battery | Unlo...,,4.4 out of 5 stars,"1,681 ratings",Not Available
3,Moto G stylus | 2021 | 2-Day battery | Unlocke...,,4.4 out of 5 stars,"2,125 ratings",In Stock.
4,Google Pixel 7-5G Android Phone - Unlocked Sma...,,4.2 out of 5 stars,123 ratings,Not Available
5,"Tracfone Alcatel TCL A3, 32GB, Black - Prepaid...",,4.0 out of 5 stars,953 ratings,In Stock.
6,OnePlus Nord N200 | 5G Unlocked Android Smartp...,,4.1 out of 5 stars,"1,714 ratings",In Stock.
7,"UMIDIGI Rugged Unlocked Cell Phones, Bison 2 P...",,4.0 out of 5 stars,107 ratings,Not Available
8,"TCL 10L, Unlocked Android Smartphone, 256GB+6G...",,4.2 out of 5 stars,"4,912 ratings",Not Available
9,"SAMSUNG Galaxy A13 LTE Cell Phone, Factory Unl...",,4.1 out of 5 stars,200 ratings,In Stock.
